In [1]:
import os
!pip install tqdm
!pip install sentence-transformers
!pip install -U torch
!pip install PyMuPDF
!pip install accelerate
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import shutil

pdf_drive_path = '/content/drive/MyDrive/RAG_model/RAG_model.pdf'
pdf_path ='./RAG_model.pdf'

if not os.path.exists(pdf_drive_path):
    print("File doesn't exist in Google Drive.")
else:
    shutil.copy(pdf_drive_path, pdf_path)
    print(f"File copied to Colab environment as {pdf_path}")

if os.path.exists(pdf_path):
    print(f"File {pdf_path} exists in Colab.")
else:
    print("File copy failed.")

File copied to Colab environment as ./RAG_model.pdf
File ./RAG_model.pdf exists in Colab.


In [5]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number+1,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:1]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 2834,
  'page_word_count': 485,
  'page_sentence_count_raw': 17,
  'page_token_count': 708.5,
  'text': 'The Rise of Alessandro Vargas – Manchester United’s New Star  In the summer of 2027, Manchester United shocked the football world by signing a relatively  unknown but immensely talented young midfielder, Alessandro Vargas, from Sporting  Lisbon for a fee of £55 million. Born in Porto but of Chilean descent, the 21-year-old had  only two seasons of professional football under his belt before the Red Devils swooped in to  secure his signature. While his name wasn’t widely known at the time, insiders at Old  Trafford were convinced that Vargas was destined to become the future heartbeat of the  Manchester United midfield.  The Announcement  The club’s official website announced the signing in early August, with a glowing  endorsement from Erik ten Hag: “Alessandro Vargas is one of the most exciting young  midfielders in Europe. His intelligence 

In [6]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,2834,485,17,708.50,The Rise of Alessandro Vargas – Manchester Uni...
1,2,2471,440,18,617.75,before scoring the winning goal himself—a curl...
2,3,2627,457,19,656.75,Fernandes and Marcus Rashford entering their p...
3,4,2793,484,23,698.25,Vargas brought a new dimension to Chile’s midf...
4,5,2774,476,20,693.50,"armband on his sleeve, Vargas would lead Unite..."


In [7]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,13.00,13.00,13.00,13.00,13.00,13.00
mean,7.00,2760.31,484.46,21.46,690.08,23.15
std,3.89,166.43,31.81,2.76,41.61,3.44
min,1.00,2471.00,435.00,17.00,617.75,17.00
25%,4.00,2627.00,457.00,19.00,656.75,20.00
50%,7.00,2793.00,485.00,23.00,698.25,24.00
75%,10.00,2898.00,509.00,23.00,724.50,26.00
max,13.00,2964.00,536.00,26.00,741.00,27.00


In [9]:
num_sentence_chunk_size = 10

def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/13 [00:00<?, ?it/s]

In [10]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,13.00,13.00,13.00,13.00,13.00,13.00,13.00
mean,7.00,2760.31,484.46,21.46,690.08,23.15,2.69
std,3.89,166.43,31.81,2.76,41.61,3.44,0.48
min,1.00,2471.00,435.00,17.00,617.75,17.00,2.00
25%,4.00,2627.00,457.00,19.00,656.75,20.00,2.00
50%,7.00,2793.00,485.00,23.00,698.25,24.00,3.00
75%,10.00,2898.00,509.00,23.00,724.50,26.00,3.00
max,13.00,2964.00,536.00,26.00,741.00,27.00,3.00


In [11]:
import re
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        pages_and_chunks.append(chunk_dict)
len(pages_and_chunks)

  0%|          | 0/13 [00:00<?, ?it/s]

35

In [12]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,35.00,35.00,35.00,35.00
mean,7.26,1011.26,166.57,252.81
std,3.56,374.98,60.39,93.74
min,1.00,131.00,20.00,32.75
25%,4.50,894.50,154.00,223.62
50%,7.00,1109.00,183.00,277.25
75%,10.00,1216.50,195.00,304.12
max,13.00,1621.00,262.00,405.25


In [13]:
min_token_length = 30
filtered_df = df[df["chunk_token_count"] <= min_token_length]
if not filtered_df.empty:
    sample_size = min(5, len(filtered_df))
    for row in filtered_df.sample(sample_size).iterrows():
        print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')
else:
    print("No rows found with token count less than or equal to 30.")


No rows found with token count less than or equal to 30.


In [14]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:1]

[{'page_number': 1,
  'sentence_chunk': 'The Rise of Alessandro Vargas – Manchester United’s New Star In the summer of 2027, Manchester United shocked the football world by signing a relatively unknown but immensely talented young midfielder, Alessandro Vargas, from Sporting Lisbon for a fee of £55 million. Born in Porto but of Chilean descent, the 21-year-old had only two seasons of professional football under his belt before the Red Devils swooped in to secure his signature. While his name wasn’t widely known at the time, insiders at Old Trafford were convinced that Vargas was destined to become the future heartbeat of the Manchester United midfield. The Announcement The club’s official website announced the signing in early August, with a glowing endorsement from Erik ten Hag: “Alessandro Vargas is one of the most exciting young midfielders in Europe. His intelligence on the ball, work ethic, and ability to control the tempo of the game make him an ideal fit for our long-term projec

In [15]:
import time
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
%%time
embedding_model.to("cuda")
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/35 [00:00<?, ?it/s]

CPU times: user 2.47 s, sys: 261 ms, total: 2.74 s
Wall time: 2.09 s


In [17]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [18]:
%%time
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True)

text_chunk_embeddings

CPU times: user 295 ms, sys: 13.4 ms, total: 308 ms
Wall time: 235 ms


tensor([[-0.0015,  0.0140,  0.0276,  ...,  0.0190,  0.0263, -0.0007],
        [-0.0129, -0.0197,  0.0362,  ...,  0.0260,  0.0270, -0.0042],
        [ 0.0005,  0.0170,  0.0020,  ...,  0.0383,  0.0271, -0.0262],
        ...,
        [-0.0257, -0.0037,  0.0217,  ...,  0.0358,  0.0200, -0.0557],
        [-0.0157,  0.0223,  0.0222,  ...,  0.0402,  0.0101, -0.0224],
        [ 0.0098,  0.0393,  0.0154,  ...,  0.0572,  0.0501, -0.0212]],
       device='cuda:0')

In [19]:
import pandas as pd

text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "/content/drive/MyDrive/RAG_model/text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)
print(f"File saved to: {embeddings_df_save_path}")

File saved to: /content/drive/MyDrive/RAG_model/text_chunks_and_embeddings_df.csv


In [20]:
colab_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(colab_save_path, index=False)
print(f"File saved to Colab environment: {colab_save_path}")

File saved to Colab environment: text_chunks_and_embeddings_df.csv


In [21]:
text_chunks_and_embedding_df_load = pd.read_csv(colab_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,The Rise of Alessandro Vargas – Manchester Uni...,1621,262,405.25,[-1.50107441e-03 1.40075926e-02 2.75763553e-...
1,1,The fans at Old Trafford erupted with applause...,1174,185,293.50,[-1.28770219e-02 -1.96580645e-02 3.62442620e-...
2,2,before scoring the winning goal himself—a curl...,1395,230,348.75,[ 5.24090428e-04 1.70429684e-02 2.03978666e-...
3,2,The system would retrieve archived interviews ...,1039,174,259.75,[-1.30381184e-02 4.42261435e-02 1.46093797e-...
4,3,Fernandes and Marcus Rashford entering their p...,1213,194,303.25,[-1.96144246e-02 1.31419776e-02 1.79680102e-...


In [22]:
import random
import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([35, 768])

In [23]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,The Rise of Alessandro Vargas – Manchester Uni...,1621,262,405.25,"[-0.00150107441, 0.0140075926, 0.0275763553, -..."
1,1,The fans at Old Trafford erupted with applause...,1174,185,293.50,"[-0.0128770219, -0.0196580645, 0.036244262, -0..."
2,2,before scoring the winning goal himself—a curl...,1395,230,348.75,"[0.000524090428, 0.0170429684, 0.00203978666, ..."
3,2,The system would retrieve archived interviews ...,1039,174,259.75,"[-0.0130381184, 0.0442261435, 0.0146093797, -0..."
4,3,Fernandes and Marcus Rashford entering their p...,1213,194,303.25,"[-0.0196144246, 0.0131419776, 0.0179680102, -0..."


In [24]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):

    query_embedding = model.encode(query,
                                   convert_to_tensor=True)
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores,
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)

    print(f"Query: {query}\n")
    print("Results:")
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")

In [25]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [26]:
from time import perf_counter as timer
from sentence_transformers import util
query = "Champions league"

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

[INFO] Time taken to get scores on 35 embeddings: 0.00648 seconds.


(tensor([0.4267, 0.4132, 0.3838, 0.3668, 0.3635], device='cuda:0'),
 tensor([24, 31,  6, 32, 14], device='cuda:0'))

In [27]:
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

[INFO] Time taken to get scores on 35 embeddings: 0.00008 seconds.
Query: Champions league

Results:
Score: 0.4267
European Glory Beckons Again With domestic success under his belt, Vargas set
his sights on the Champions League. The competition had always held a special
place in his heart, and now, as a manager, he was determined to bring the trophy
back to Old Trafford. In the 2047/48 season, United reached the semi-finals of
the Champions League, facing off against a dominant Paris Saint-Germain side. In
the first leg, United drew 1-1 at home, setting up a tense return leg in Paris.
With the pressure mounting, Vargas delivered a masterclass in tactical
preparation. His midfield, marshaled by one of his young protégés, controlled
the game from start to finish. A 2-0 victory in Paris sent United to the final,
where they would face Bayern Munich—a rematch of the 2032 final Vargas had won
as a player. The final was a tight, cagey affair. Vargas, known for his calm
demeanor, made key subs

In [28]:
import torch
# gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
# gpu_memory_gb = round(gpu_memory_bytes / (2**30))
# print(f"Available GPU memory: {gpu_memory_gb} GB")

In [29]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
# if gpu_memory_gb < 5.1:
#     print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
# elif gpu_memory_gb < 8.1:
#     print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
#     use_quantization_config = True
#     model_id = "google/gemma-2b-it"
# elif gpu_memory_gb < 19.0:
#     print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
#     use_quantization_config = False
#     model_id = "google/gemma-2b-it"
# elif gpu_memory_gb > 19.0:
# print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")

use_quantization_config = False
model_id = "google/gemma-7b-it"

# print(f"use_quantization_config set to: {use_quantization_config}")
# print(f"model_id set to: {model_id}")

In [30]:
%cd /content/drive/MyDrive/RAG_model
from my_key import my_key2
from huggingface_hub import login
login(token=my_key2)

/content/drive/MyDrive/RAG_model
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")
model_id = model_id
print(f"[INFO] Using model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16,
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=False,
                                                 attn_implementation=attn_implementation)

if not use_quantization_config:
    llm_model.to("cuda")

[INFO] Using attention implementation: flash_attention_2
[INFO] Using model_id: google/gemma-7b-it


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [32]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaFlashAttention2(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps

In [33]:
# def get_model_num_params(model: torch.nn.Module):
#     return sum([param.numel() for param in model.parameters()])

# get_model_num_params(llm_model)

8537680896

In [ ]:
# def get_model_mem_size(model: torch.nn.Module):
#     mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
#     mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])
#     model_mem_bytes = mem_params + mem_buffers
#     model_mem_mb = model_mem_bytes / (1024**2)
#     model_mem_gb = model_mem_bytes / (1024**3)

#     return {"model_mem_bytes": model_mem_bytes,
#             "model_mem_mb": round(model_mem_mb, 2),
#             "model_mem_gb": round(model_mem_gb, 2)}

# get_model_mem_size(llm_model)

{'model_mem_bytes': 17075376128,
 'model_mem_mb': 16284.35,
 'model_mem_gb': 15.9}

In [34]:
input_text = "Who won the champions league"
print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
Who won the champions league

Prompt (formatted):
<bos><start_of_turn>user
Who won the champions league<end_of_turn>
<start_of_turn>model



In [35]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")
outputs = llm_model.generate(**input_ids,
                             max_new_tokens=256)
print(f"Model output (tokens):\n{outputs[0]}\n")

Model input (tokenized):
{'input_ids': tensor([[    2,     2,   106,  1645,   108,  6571,  2792,   573, 17743, 15296,
           107,   108,   106,  2516,   108]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Model output (tokens):
tensor([     2,      2,    106,   1645,    108,   6571,   2792,    573,  17743,
         15296,    107,    108,    106,   2516,    108, 235285,    749,    780,
           791,   3684,    577,   1879, 235290,   1602,   2113, 235269,   5852,
           590,   2952,   3448,    573,   2872,    576,   1064,   2792,    573,
         17743,  15296,   3646, 235265,    590,   5656,  18604,    671,   5613,
          4303,    604,    573,   1546,    908, 235290,    511, 235290,   1545,
          2113, 235265,      1], device='cuda:0')

CPU times: user 2.04 s, sys: 91.9 ms, total: 2.14 s
Wall time: 2.32 s


In [36]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<bos><bos><start_of_turn>user
Who won the champions league<end_of_turn>
<start_of_turn>model
I do not have access to real-time information, therefore I cannot answer the question of who won the champions league today. I recommend checking an official source for the most up-to-date information.<eos>



In [37]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

Input text: Who won the champions league

Output text:
I do not have access to real-time information, therefore I cannot answer the question of who won the champions league today. I recommend checking an official source for the most up-to-date information.


In [38]:
manual_questions = [
    "Who is Alessandro Vargas?",
    "Which team did Alessandro Vargas played for?",
]
query_list =manual_questions

In [39]:
import random
query = random.choice(query_list)
print(f"Query: {query}")
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

Query: Which team did Alessandro Vargas played for?
[INFO] Time taken to get scores on 35 embeddings: 0.00008 seconds.


(tensor([0.7000, 0.6224, 0.6171, 0.6144, 0.6093], device='cuda:0'),
 tensor([20, 12, 15, 16,  5], device='cuda:0'))

In [40]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:

    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What is offside in football?
Answer: Offside in football occurs when an attacking player is nearer to the opponent's goal line than both the ball and the second-last defender when the ball is played to them. The offside rule is in place to prevent players from lingering near the goal and gaining an unfair advantage. However, a player is not offside if they are in their own half of the field or level with the second-last defender at the moment the ball is played.
\nExample 2:
Query: How long is a football match?
Answer: A standard football match lasts 90 minutes, divided into two halves of 45 minutes each, with a 15-minute halftime break in between. In some cases, such as knockout stages in tournaments, extra time or penalties may be played if the match is tied after the regular 90 minutes.
\nExample 3:
Query: What is a hat-trick in football?
Answer: A hat-trick in football refers to a player scoring three goals in a single match. Achieving this is a significant accomplishment and is often celebrated as a milestone in a player's career. The goals do not need to be consecutive, and they can be scored at any point during the match.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""

    base_prompt = base_prompt.format(context=context, query=query)
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [41]:
query = random.choice(query_list)
print(f"Query: {query}")
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: Which team did Alessandro Vargas played for?
[INFO] Time taken to get scores on 35 embeddings: 0.00010 seconds.
<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What is offside in football?
Answer: Offside in football occurs when an attacking player is nearer to the opponent's goal line than both the ball and the second-last defender when the ball is played to them. The offside rule is in place to prevent players from lingering near the goal and gaining an unfair advantage. However, a player is not offside if they are in their own half of the field or level with the second-last defender at the moment the ball is played.

Example 2:
Query: How long 

In [42]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = llm_model.generate(**input_ids,
                             temperature=0.5,
                             do_sample=True,
                             max_new_tokens=256)

output_text = tokenizer.decode(outputs[0])
print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")

Query: Which team did Alessandro Vargas played for?
RAG answer:
<bos>Sure, here is the answer to the query:

Alessandro Vargas played for Manchester United. He joined the club in 2021 and achieved great success, winning the Premier League and the Champions League. He was also named Man of the Match in the Champions League final.<eos>
CPU times: user 2.86 s, sys: 13 ms, total: 2.88 s
Wall time: 2.89 s


In [43]:
def ask(query,
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):

    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)

    context_items = [pages_and_chunks[i] for i in indices]
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu()
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")
    if return_answer_only:
        return output_text

    return output_text, context_items

In [44]:
query = random.choice(query_list)
print(f"Query: {query}")
answer, context_items = ask(query=query,
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Query: Who is Alessandro Vargas?
[INFO] Time taken to get scores on 35 embeddings: 0.00009 seconds.
Answer:

**Answer:**  Alessandro Vargas is a legendary former footballer who played for
Manchester United. He is known for his exceptional skills, leadership, and
inspiration. Born in Portugal, he expressed a deep connection to his Chilean
heritage and played for Chile in a World Cup Qualifier. Throughout his career,
Vargas has left an indelible mark on the game, and his story continues to be
celebrated as one of the most inspiring in football history.
Context items:


[{'page_number': 8,
  'sentence_chunk': "Players gravitated toward him, seeing him not just as a coach, but as a living legend whose career they had watched in awe. Vargas's work began to pay off. By 2044, several young players from the academy had broken into the first team, and many credited Vargas with their rapid development. One such player, a young playmaker named Diego Morales, even drew comparisons to Vargas himself. In an interview, Morales said: “Working with Alessandro has been incredible. He taught me",
  'chunk_char_count': 477,
  'chunk_word_count': 80,
  'chunk_token_count': 119.25,
  'embedding': array([ 5.58943301e-03,  1.67987719e-02,  3.14618764e-03, -1.18262423e-02,
          7.85618499e-02, -6.42084181e-02, -2.15222947e-02,  1.78391598e-02,
         -4.22259681e-02,  3.85235995e-02,  2.90496629e-02,  3.84518653e-02,
          7.11394101e-02, -1.13648176e-02, -2.28796396e-02,  1.99620379e-03,
         -2.19142623e-02, -2.79166512e-02, -2.87910681e-02, -2.22658385e-0

In [ ]:
# from git_setup import configure_git, change_directory, create_gitignore, initialize_repo, add_files, commit_files, push_to_github
# from my_key import my_key

# configure_git("Irfan-Hamid", "irfanhamid19@gmail.com")
# change_directory()
# create_gitignore()
# initialize_repo()
# add_files()
# commit_files("Initial commit: Added notebook and PDF files")
# push_to_github(my_key, 'github.com/your_github_username/your_repo_name.git')


In [ ]:
%cd /content/drive/MyDrive/RAG_model
from git_setup import configure_git, change_directory, create_gitignore, initialize_repo, add_files, commit_files, push_to_github
from my_key import my_key

/content/drive/MyDrive/RAG_model


In [ ]:
configure_git("Irfan-Hamid", "irfanhamid19@gmail.com")
add_files()
commit_files("fifth commit: code updation")
push_to_github(my_key, 'github.com/Irfan-Hamid/LLM_RAG_IMPLEMENTATION.git')

In [ ]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   LLM_RAG_IMPLEMENT.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
